# *__Working on BTCUSD predictions with GRU model.__*

## *__Check first before starting__*

In [19]:
# Do 'pipenv install ipykernel' if you get error.
print("Kernel is working correctly!")

Kernel is working correctly!


In [20]:
import os

# Change the working directory to the project root
Working_directory = os.path.normpath("C:/Users/gilda/OneDrive/Documents/_NYCU/MASTER_S_studies/Master\'s Thesis/LABORATORY/_Global_Pytorch/Continual_Learning")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")  # Prints the current working directory

Working directory: C:\Users\gilda\OneDrive\Documents\_NYCU\MASTER_S_studies\Master's Thesis\LABORATORY\_Global_Pytorch\Continual_Learning


## **__All imports__**

In [21]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume
import math
from scipy.ndimage import gaussian_filter1d
from typing import Callable, Tuple
import shutil
import contextlib
import traceback
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## __**All functions (For data processing)**__

In [22]:
def ensure_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def plot_with_matplotlib(data: pd.DataFrame, 
                         title: str, 
                         interactive: bool = False, 
                         save_path: str = None, 
                         show_plot: bool = True, 
                         save_matplotlib_object: str = None) -> None:
    """
    Plot data using Matplotlib, with optional interactivity using mpl-interactions.
    
    Parameters:
    - data (pd.DataFrame): The data to plot, must contain 'close' column.
    - title (str): The title of the plot.
    - interactive (bool): If True, enables interactive zoom and pan.
    - save_path (Optional[str]): If provided, saves the plot to this path.
    - show_plot (bool): If True, displays the plot. If False, skips display.
    - save_matplotlib_object (Optional[str]): If provided, saves the Matplotlib figure object to this file path.
    """
    if not all(col in data.columns for col in ['close']):
        raise ValueError("The input DataFrame must contain 'close' column.")

    # Use the default Matplotlib color cycle for the line
    default_blue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
    print(f"Default blue: {default_blue}")

    # Explicit colors for trends
    trend_colors = {
        0: 'black',
        1: 'yellow',
        2: 'red',
        3: 'green',
        4: default_blue #'purple',
    }
    # unique_trends = [0, -25, -15, 15, 25]
    # colormap = plt.cm.get_cmap('tab10', len(unique_trends))  # Choose 'tab10' or 'Set1' for distinct colors
    # trend_colors = {trend: colormap(i) for i, trend in enumerate(unique_trends)}

    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot data as a single connected line, colored by trend
    if 'trend' in data.columns:
        legend_added = set() # Track which trends have already been added to the legend
        prev_idx = data.index[0]
        for idx, row in data.iterrows():
            if idx != prev_idx:
                trend_key = int(row['trend'])  # Convert trend value to int for lookup
                label = f'Trend {trend_key}' if trend_key not in legend_added else None
                ax.plot([prev_idx, idx], 
                        [data.loc[prev_idx, 'close'], row['close']],
                        color=trend_colors[trend_key], 
                        linestyle='-', 
                        # marker='o', 
                        linewidth=1,
                        label=label  # Add label only if it's not in the legend
                )
                legend_added.add(trend_key)  # Mark this trend as added to the legend
            prev_idx = idx

        ax.set_title(f"{title} (Connected, Colored by Trend)")
    else:
        # Default plot if no trend column exists
        ax.plot(data.index, data['close'], label='Closing Price', linestyle='-', marker='o', 
                markersize=2, linewidth=1, color=default_blue, markerfacecolor='green', markeredgecolor='black')
        ax.set_title(title)
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Closing Price (USD)')
    
    # Add a legend manually for trends
    # for trend, color in trend_colors.items():
    #     ax.plot([], [], color=color, label=f'Trend {trend}')
    ax.legend()
    ax.grid()
    
    # Enable interactivity if requested
    if interactive:
        zoom_factory(ax)  # Enable zoom with mouse wheel
        panhandler(fig)   # Enable panning with left-click
        print("Interactive mode enabled. Use mouse wheel to zoom and left click to pan.")

    # Save the plot if a path is provided
    if save_path:
        fig.tight_layout()  # Ensures the layout is clean
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")

    # Save the Matplotlib figure object
    if save_matplotlib_object:
        with open(save_matplotlib_object, 'wb') as f:
            pickle.dump(fig, f)
        print(f"Matplotlib figure object saved to: {save_matplotlib_object}")

    if show_plot:
        plt.show()
    else:
        print("Plot display skipped.")

def load_and_show_pickle(pickle_file_path: str):
    """
    Load a pickled Matplotlib figure object and display it with optional interactivity.

    Parameters:
    - pickle_file_path (str): Path to the pickled Matplotlib figure file.

    Returns:
    - None
    """
    try:
        # Open the pickle file and load the figure
        with open(pickle_file_path, "rb") as f:
            loaded_fig = pickle.load(f)

        print(f"Figure successfully loaded and displayed from: {pickle_file_path}")

        # Use plt.show() to allow interactivity
        plt.show(block=True)

    except FileNotFoundError:
        print(f"Error: File not found at {pickle_file_path}. Please check the path.")
    except Exception as e:
        print(f"Error loading the pickled figure: {e}")

# Save data to CSV
def save_to_csv(df: pd.DataFrame, file_path: str):
    """
    Save a DataFrame to a CSV file.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data to be saved.
        file_path (str): The file path (including the file name) to save the CSV.

    Returns:
        None
    """
    df.to_csv(file_path)
    # df_to_save = df.copy()
    # df_to_save["date"] = df_to_save.index.strftime('%Y-%m-%d %H:%M:%S')  # Add formatted index as a column
    
    # Save the DataFrame to CSV
    # df_to_save.to_csv(file_path)
    print(f"\nSuccessfully saved data with moving average to CSV: \n\t{file_path}\n")

def read_csv_file(file_path: str, preview_rows: int = 5, 
                  days_towards_end: int = None, 
                  days_from_start: int = None, description: str = ""):
    """
    Reads a CSV file and returns a pandas DataFrame filtered by date range.

    Args:
        file_path (str): The path to the CSV file.
        preview_rows (int): Number of rows to preview (default is 5).
        days_towards_end (int, optional): Number of days from the most recent date to retrieve data.
        days_from_start (int, optional): Number of days from the oldest date of the filtered data to retrieve data.
        description (str): A brief description of the dataset being loaded.
                           Explanation:
                           - To retrieve data from the **end**: Use `days_towards_end`.
                           - To retrieve data from the **start of the filtered range**: Use `days_from_start`.
                           - To retrieve data from the **middle**: Use both:
                             For example, if `days_towards_end=100` and `days_from_start=50`,
                             the function will first filter the last 100 days of the dataset,
                             and then filter the first 50 days from this range.
                             This results in data between the last 100th and the last 50th day.

    Returns:
        DataFrame: The loaded and filtered data from the CSV file.
    """
    try:
        if description:
            print(f"\nDescription: {description}")
        print(f"\nFile path: {file_path}")
        
        # Read the CSV file
        data = pd.read_csv(file_path, parse_dates=['date'], index_col='date')
        
        # Filter by days towards the end
        if days_towards_end is not None:
            last_date = data.index.max()  # Get the most recent date in the dataset
            end_cutoff_date = last_date - pd.Timedelta(days=days_towards_end)
            data = data[data.index >= end_cutoff_date]
            print(f"\nRetrieving data from the past {days_towards_end} days (from {end_cutoff_date.date()} onwards):")
        
        # Filter by days from the start (from the filtered data)
        if days_from_start is not None:
            first_date = data.index.min()  # Get the earliest date in the filtered dataset
            start_cutoff_date = first_date + pd.Timedelta(days=days_from_start)
            data = data[data.index <= start_cutoff_date]
            print(f"\nRetrieving the first {days_from_start} days from the filtered data (up to {start_cutoff_date.date()}):")

        if preview_rows:
            # Print a preview of the data
            print(f"\nPreview of the first {preview_rows} rows:")
            # print(data.head(preview_rows), '\n')
            display(data.head(preview_rows))
            print()

            print(f"\nPreview of the last {preview_rows} rows:")
            # print(data.tail(preview_rows), '\n')
            display(data.tail(preview_rows))
            print()

        return data
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
    except pd.errors.ParserError:
        print("Error: The file could not be parsed. Please check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def downsample_minute_data(data: pd.DataFrame, n: int) -> pd.DataFrame:
    """
    Downsample minute data into N-minute intervals by retaining every Nth row.

    Parameters:
        data (pd.DataFrame): The original DataFrame with a datetime index.
        n (int): The number of minutes for the downsampling interval.

    Returns:
        pd.DataFrame: Downsampled DataFrame.
    """
    print("\n========---> Downsampling the data! \n")
    data = data.copy()
    # Ensure the index is a DatetimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        try:
            data.index = pd.to_datetime(data.index)  # Convert to DatetimeIndex
        except Exception as e:
            raise ValueError("The DataFrame index could not be converted to DatetimeIndex.") from e

    # Filter rows where the minute index modulo N is 0
    downsampled_data = data[data.index.minute % n == 0]

    return downsampled_data

def add_indicators(data: pd.DataFrame, output_path: str, number_days: int = None, preview_rows: int = 5, freq: str = '1min') -> None:
    """
    Adds technical indicators to a financial dataset, trims the dataset to the last N days, 
    and saves the enriched dataset to a CSV file.

    Args:
        data (pd.DataFrame): The input financial dataset. Must contain at least the following columns: 
                             'open', 'high', 'low', 'close', and 'volume'. The index should be a datetime index.
        output_path (str): The file path where the enriched dataset will be saved as a CSV file.
        number_days (int): The number of days to retain from the most recent date in the dataset.
        preview_rows (int, optional): Number of rows to preview during various stages of processing. Default is 5.
        freq (str): Frequency for the continuous time index. Default is '1min'.

    Returns:
        None: The function saves the processed dataset to a file and does not return a value.
    """
    try:
        print("\n========---> Adding indicators to the data!")
        if number_days is not None:
            # Trim data to the last 190 days
            last_date = data.index.max()  # Get the most recent date in the dataset
            cutoff_date = last_date - pd.Timedelta(days=number_days)  # Calculate the cutoff date
            data = data[data.index >= cutoff_date]  # Keep only rows within the last 190 days
            print(f"\nData trimmed to the last {number_days} days from {cutoff_date} to {last_date}.\n")

        # Sort data in ascending order by date for proper indicator calculations
        data = data.sort_index(ascending=True)
        print(f"\nPreview of the first {preview_rows} rows after reversing: \n")
        display(data.head(preview_rows))

        # Verify the presence of required columns
        required_columns = ['open', 'high', 'low', 'close', 'volume']
        if not all(col in data.columns for col in required_columns):
            raise ValueError("Dataset is missing required columns: 'open', 'high', 'low', 'close', 'volume'.")

        #-----------------------------------------------------------------
        # Get missing timestamps
        missing_timestamps = pd.date_range(
            start=data.index.min(), # Returns smallest/earliest/oldest date
            end=data.index.max(),
            freq=freq,  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
            tz=data.index.tz,
        ).difference(data.index)

        # Generate a continuous time index (1-minute frequency)
        full_time_index = pd.date_range(
            start=data.index.min(),
            end=data.index.max(),
            freq=freq, 
            tz=data.index.tz,
        )
        index_name = data.index.name
        
        # Reindex the DataFrame to include all timestamps, introducing NaNs for missing timestamps
        data = data.reindex(full_time_index)
        data.index.name = index_name  # Restore index name after reindexing

        # Fill missing data with the previous available value
        data = data.ffill()  # Forward-fill missing values

        # Check if there are missing timestamps
        if missing_timestamps.empty:
            print("\nNo missing timestamps.\n")
        else:
            for timestamp in missing_timestamps:
                print(f"\nMissing timestamp: {timestamp}")
                try:
                    # Safely access the row after reindexing and forward-filling
                    print(f"Data at missing timestamp ({timestamp}):")
                    print(data.loc[timestamp], '\n')
                except KeyError:
                    print(f"No data available for timestamp {timestamp}\n")

        # Get missing timestamps
        missing_timestamps = pd.date_range(
            start=data.index.min(), # Returns smallest/earliest/oldest date
            end=data.index.max(),
            freq=freq,  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
            tz=data.index.tz,
        ).difference(data.index)
        print(f"\nMissing timestamps time: \n{missing_timestamps}\n")
        #-----------------------------------------------------------------


        # Drop unnecessary columns if present
        columns_to_drop = ['volume weighted average', 'barCount']  # Adjust if needed
        data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors='ignore')

        # Add Technical Indicators

        ## Trend Indicators
        # Simple Moving Averages (SMA)
        data['SMA_5'] = trend.sma_indicator(data['close'], window=5)
        data['SMA_10'] = trend.sma_indicator(data['close'], window=10)
        data['SMA_50'] = trend.sma_indicator(data['close'], window=50)

        # Exponential Moving Average (EMA)
        data['EMA_10'] = trend.ema_indicator(data['close'], window=10)

        # Moving Average Convergence Divergence (MACD)
        data['MACD'] = trend.macd(data['close'])
        data['MACD_signal'] = trend.macd_signal(data['close'])

        # Bollinger Bands
        data['BB_upper'] = volatility.bollinger_hband(data['close'], window=20, window_dev=2)
        data['BB_middle'] = volatility.bollinger_mavg(data['close'], window=20)
        data['BB_lower'] = volatility.bollinger_lband(data['close'], window=20)

        ## Momentum Indicators
        # Relative Strength Index (RSI)
        data['RSI_14'] = momentum.rsi(data['close'], window=14)

        # Rate of Change (ROC)
        data['ROC_10'] = momentum.roc(data['close'], window=10)

        ## Volume Indicators
        # On-Balance Volume (OBV)
        data['OBV'] = volume.on_balance_volume(data['close'], data['volume'])

        # Accumulation/Distribution Line (A/D Line)
        data['AD_Line'] = volume.acc_dist_index(data['high'], data['low'], data['close'], data['volume'])

        ## Volatility Indicators
        # Average True Range (ATR)
        data['ATR_14'] = volatility.average_true_range(data['high'], data['low'], data['close'], window=14)

        # Bollinger Band Width (BBW)
        data['BBW'] = (data['BB_upper'] - data['BB_lower']) / data['BB_middle']

        # Adjust decimal precision as needed
        data = data.round(decimals=9) 

        # Replace 0 values with NaN to allow forward-fill
        data = data.replace(0, pd.NA)
        
        # Forward-fill to replace NaN (originally 0) with the previous row value
        data = data.ffill()

        # Drop rows with NaN values (optional, as technical indicators often result in NaN for initial (oldest dates) rows)
        data = data.dropna()

        # Sort the dataset back to descending order for final output
        data = data.sort_index(ascending=False)

        print(f"\nPreview of the first {preview_rows} rows with indicators: \n")
        display(data.head(preview_rows))

        print(f"\nPreview of the last {preview_rows} rows with indicators: \n")
        display(data.tail(preview_rows))

        # Save the enriched dataset
        data.to_csv(output_path)
        print(f"\nEnriched dataset saved to \n\t{output_path}\n")

    except Exception as e:
        print(f"\nError adding indicators: {e}\n")
        print("\nDetailed traceback:")
        traceback.print_exc()

# Function to calculate percentage changes
def calculate_percentage_changes(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate percentage changes for a given dataset and return the result in ascending order.
    The `real_close` column is added after the percentage changes calculation.

    Parameters:
        data (pd.DataFrame): The input dataset with a datetime index and numeric values (e.g., closing prices).
                              The DataFrame must already have a properly formatted datetime index.

    Returns:
        pd.DataFrame: A DataFrame containing the percentage changes, indexed by date in ascending order.
                      The 'real_close' column is added, holding the original close prices.
    """
    data = data.sort_index(ascending=True).copy()  # Ensure chronological order
    data_pct_change = data.pct_change() # Calculate percentage changes
    data_pct_change['real_close'] = data['close'] # Add the 'real_close' column (copy of original 'close' column)
    return data_pct_change.dropna()  # Drop rows with NaN values resulting from pct_change()

def calculate_log_returns_all_columns(data: pd.DataFrame, exclude_columns: list = [], dropna: bool = True) -> pd.DataFrame:
    """
    Calculate log returns for all numeric columns in a pandas DataFrame,
    excluding specified columns, and removing excluded columns from the returned DataFrame.

    Args:
        data (pd.DataFrame): Input DataFrame containing numeric data.
        exclude_columns (list): List of columns to exclude from log return calculations and the result.
        dropna (bool): Whether to drop rows with NaN values resulting from the calculation.

    Returns:
        pd.DataFrame: DataFrame with log returns for numeric columns,
                      excluding specified columns.
    """
    data = data.copy().drop(columns=exclude_columns)
    columns_to_transform = data.select_dtypes(include=[np.number]).columns
    print(f"columns_to_transform = \n{columns_to_transform}, \nlen(columns_to_transform) = {len(columns_to_transform)}")

    for col in columns_to_transform:
        # Ensure no negative or zero values
        if (data[col] <= 0).any():
            raise ValueError(f"Column '{col}' contains non-positive values. Log returns require strictly positive values.")
        data[col] = np.log(data[col] / data[col].shift(1))

    # Optionally drop rows with NaN values
    return data.dropna() if dropna else data

# Prepare the data for the model by creating sequences of input features and output targets.
def create_sequences(data: pd.DataFrame, 
                     input_length: int, 
                     output_length: int, 
                     sliding_interval: int = 60) -> tuple[np.ndarray, np.ndarray, pd.Index]:
    """
    Generate input-output sequences with a sliding window.

    Parameters:
        data (pd.DataFrame): DataFrame containing features for time-series modeling, indexed by 'date'.
        input_length (int): Number of timesteps for input sequences (e.g., 2880 for 2 days).
        output_length (int): Number of timesteps for output sequences (e.g., 1440 for 1 day).
        sliding_interval (int): Interval to slide the window (e.g., 60 for 1 hour).

    Returns:
        tuple[np.ndarray, np.ndarray, pd.Index, list, list]:
            - X: Input sequences of shape (num_sequences, input_length, num_features).
            - y: Output sequences of shape (num_sequences, output_length, num_features).
            - indices: A pandas Index object with the start dates of each input sequence for reference.
            - end_start_X: The real end values for each input sequence.
    """
    data = data.copy()
    # Preserve the date index for later reference
    original_index = data.index
    # print(original_index[:5])

    # Reset index to allow slicing by integer position
    data = data.reset_index(drop=True)  # Removes the 'date' column which was the index
    print("\nColumns after resetting index with drop=True: \n", data.columns)
    close_column_index = data.columns.get_loc('close')
    print(f"'close' column index: {close_column_index}")
    num_columns = data.shape[1]
    print(f"Number of columns: {num_columns}")
    print('')
    
    X, y, indices = [], [], []
    real_close_X = []
    total_length = len(data)

    for start in range(0, total_length - input_length - output_length + 1, sliding_interval):
        end_input = start + input_length
        end_output = end_input + output_length

        # Slice input and output sequences, excluding 'real_close' column
        X.append(data.iloc[start:end_input].drop(columns=['real_close']).values)
        y.append(data.iloc[end_input:end_output].values)
        
        # Save the start date of the sequence
        indices.append(original_index[start])

        # Get the real 'close' value at the start of the input and output sequences
        real_close_X.append(data.iloc[start:end_input]['real_close'])
    
    return np.array(X), np.array(y), pd.Index(indices), np.array(real_close_X)

def created_sequences_2(data: pd.DataFrame, sequence_length: int = 60, sliding_interval: int = 60) -> list:
    """
    Divide the dataset into sequences based on the sequence_length.
    Each sequence must fully cover the window size.

    Args:
    - data (pd.DataFrame): The input DataFrame.
    - sequence_length (int): The window size for sequences.

    Returns:
    - sequences (list): A list of sequences (as DataFrames).
    """
    sequences = []
    for i in range(0, len(data) - sequence_length + 1, sliding_interval):
        # print(f"Processing sequence starting at index: {i}")
        seq = data.iloc[i:i + sequence_length].copy()
        sequences.append(seq)
    # print(f"Total sequences created: {len(sequences)}")
    return sequences

def prepare_percentage_change_data(df: pd.DataFrame, 
                                   features: list, 
                                   target: str = 'close', 
                                   window_size: int = 50, 
                                   test_size: float = 0.25, 
                                   shuffle: bool = False) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Prepares percentage change stock data for training a sequence-based model.

    Parameters:
        df (pd.DataFrame): The percentage change data.
        features (list): List of column names to use as input features.
        target (str): The column name to predict.
        window_size (int): The size of the sliding window.
        test_size (float): The proportion of the dataset to include in the test split.
        shuffle (bool): Whether to shuffle the data before splitting.

    Returns:
        X_train, X_test, y_train, y_test: Prepared train and test datasets.
    """
    # Create sequences
    sequences = []
    targets = []
    for i in range(len(df) - window_size):
        sequences.append(df[features].iloc[i:i + window_size].values)
        targets.append(df[target].iloc[i + window_size])  # Predict the target at the next step
    
    sequences = np.array(sequences)
    targets = np.array(targets)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(sequences, targets, test_size=test_size, shuffle=shuffle)

    return X_train, X_test, y_train, y_test

def moving_average(data, window_size=5):
    data = data.sort_index(ascending=True).copy()
    return data.rolling(window=window_size).mean()

def exponential_moving_average(data, span=5):
    data = data.sort_index(ascending=True).copy()
    return data.ewm(span=span, adjust=False).mean()

def gaussian_smoothing(data: pd.DataFrame, sigma=2) -> pd.DataFrame:
    """
    Applies Gaussian smoothing to numeric columns in a DataFrame and ensures the index is sorted in ascending order.

    Args:
        data (pd.DataFrame): Input DataFrame.
        sigma (float): Standard deviation for the Gaussian kernel. Defaults to 2.

    Returns:
        pd.DataFrame: A DataFrame with smoothed numeric columns and sorted index.
    """
    # Sort the DataFrame by index in ascending order
    data = data.sort_index(ascending=True).copy()
    for column in data.columns:
        if pd.api.types.is_numeric_dtype(data[column]):  # Only apply to numeric columns
            data[column] = gaussian_filter1d(data[column].values, sigma=sigma)
    return data

def detect_trends(
    dataframe: pd.DataFrame, 
    column: str = 'close', 
    lower_threshold: float = 0.10, 
    upper_threshold: float = 0.20
) -> pd.DataFrame:
    """
    Processes a DataFrame to calculate log returns and categorize trends based on thresholds.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing the price data.
        column (str): Column name for price data. Defaults to 'close'.
        lower_threshold (float): Threshold for categorizing moderate trends. Defaults to 0.10.
        upper_threshold (float): Threshold for categorizing strong trends. Defaults to 0.20.

    Returns:
        pd.DataFrame: A DataFrame with added columns:
                      - 'log_return': Logarithmic returns of the specified column.
                      - 'trend': Categorized trend values:
                          - -25 for very strong negative trend
                          - -15 for moderate negative trend
                          - 0 for no trend
                          - 15 for moderate positive trend
                          - 25 for very strong positive trend
    """
    # Copy to avoid modifying the original DataFrame
    df = dataframe.copy()
    
    # Calculate log returns
    df['log_return'] = np.log(df[column] / df[column].shift(1))
    
    # Function to categorize trends
    def categorize_trend(log_return):
        if log_return < -upper_threshold:
            return -25
        elif -upper_threshold <= log_return < -lower_threshold:
            return -15
        elif -lower_threshold <= log_return <= lower_threshold:
            return 0
        elif lower_threshold < log_return <= upper_threshold:
            return 15
        else:  # log_return > upper_threshold
            return 25
    
    # Apply trend categorization
    df['trend'] = df['log_return'].apply(categorize_trend)

    # Drop NaN values caused by shift
    return df.dropna()

def detect_trends_2(
    dataframe: pd.DataFrame, 
    column: str = 'close', 
    lower_threshold: float = 0.10, 
    upper_threshold: float = 0.20
) -> pd.DataFrame:
    """
    Processes a DataFrame to calculate log returns and categorize trends using interval bins.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing the price data.
        column (str): Column name for price data. Defaults to 'close'.
        lower_threshold (float): Threshold for categorizing moderate trends. Defaults to 0.10.
        upper_threshold (float): Threshold for categorizing strong trends. Defaults to 0.20.

    Returns:
        pd.DataFrame: A DataFrame with added columns:
                      - 'log_return': Logarithmic returns of the specified column.
                      - 'trend': Categorized trend values:
                          - -25 for very strong negative trend
                          - -15 for moderate negative trend
                          - 0 for no trend
                          - 15 for moderate positive trend
                          - 25 for very strong positive trend
    """
    # Copy to avoid modifying the original DataFrame
    df = dataframe.copy()

    # Calculate log returns
    df['log_return'] = np.log(df[column] / df[column].shift(1))
    
    # Define bins and corresponding labels for trends
    bins = [-np.inf, -upper_threshold, -lower_threshold, lower_threshold, upper_threshold, np.inf]
    labels = [-25, -15, 0, 15, 25]
    
    # Categorize trends using pd.cut
    df['trend'] = pd.cut(df['log_return'], bins=bins, labels=labels, right=True)
    
    # Replace NaN trends with 0 (default value for no trend)
    df['trend'] = df['trend'].fillna(0).astype(int)

    # Drop NaN values caused by shift
    return df.dropna()

def detect_trends_3(
    dataframe: pd.DataFrame, 
    column: str = 'close', 
    lower_threshold: float = 0.001, 
    upper_threshold: float = 0.02,
    reverse_steps: int = 7
) -> pd.DataFrame:
    """
    Detects trends based on log return data provided in a specified column and categorizes them into different strength levels.

    This function analyzes time-series data by evaluating cumulative trends in log return values provided in the input DataFrame. It uses three dictionaries (`dic1`, `dic2`, `dic3`) to track different phases of trends, handles multi-step reversals, and classifies trends dynamically based on cumulative product thresholds and specified thresholds for trend strengths.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing log return data.
        column (str): Column name containing log return values. Defaults to 'close'.
        lower_threshold (float): Threshold for categorizing moderate trends. Defaults to 0.001.
        upper_threshold (float): Threshold for categorizing strong trends. Defaults to 0.02.
        reverse_steps (int): Number of consecutive steps to confirm a trend reversal. Defaults to 7.

    Returns:
        pd.DataFrame: A DataFrame with an added column:
                    - 'trend': Categorized trend values based on the detected phases:
                        - 0: No trend
                        - 1: Moderate negative trend
                        - 2: Very strong negative trend
                        - 3: Moderate positive trend
                        - 4: Very strong positive trend

    Function Details:
    1. **Input Assumption**:
    - The input DataFrame already contains log return data in the specified column (`column`).

    2. **Trend Tracking**:
    - Uses dictionaries to monitor trends:
        - `dic1`: Tracks the first phase of the trend.
        - `dic2`: Tracks the second phase if a reversal occurs.
        - `dic3`: Tracks the third phase if another reversal occurs.

    3. **Cumulative Product**:
    - Calculates the cumulative product of `(1 + log_return)` from the specified column to evaluate the strength of trends.

    4. **Reversal Handling**:
    - If a trend reversal persists beyond `reverse_steps`, labels are assigned based on the cumulative product tracked in `dic1`.
    - Subsequent reversals are merged or labeled independently if conditions are met.

    5. **Label Assignment**:
    - Labels are dynamically assigned based on cumulative product thresholds for positive and negative trends:
        - Positive trends are categorized as moderate (3) or strong (4).
        - Negative trends are categorized as moderate (1) or strong (2).

    6. **Edge Cases**:
    - Properly handles scenarios where data points are insufficient for trend analysis or when trend phases overlap, ensuring all data points are labeled.
    """
    # Copy to avoid modifying the original DataFrame
    df = dataframe.copy()
    df['trend'] = None  # Default value 
    
    # print("\n#-------------------- Working on 'trend' patterns -----------------------#")
    dic1, dic2, dic3 = None, None, None # Initialize trend tracking dictionaries
    # dic1 = None # {'ids': [], 'last_sign': None, 'cumulative': 1.0}
    
    def assign_label(dictio_, lower_threshold, upper_threshold):
        cumulative = dictio_['cumulative']
        # print(f"cumulative = {cumulative}")
        if cumulative > (1 + upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 4  # Very strong positive
        elif (1 + lower_threshold) < cumulative <= (1 + upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 3  # Moderate positive
        elif (1 - upper_threshold) < cumulative <= (1 - lower_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 1  # Moderate negative
        elif cumulative <= (1 - upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 2  # Very strong negative
        else:
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 0  # No trend
    
    #----------------------- For Loop -----------------------#
    for idx, log_ret in enumerate(df[column]):
        sign = 1 if log_ret > 0 else -1

        if dic1 is None:  # Initialize dic1
            # print(f"\nThis one time condition 'if loop' is running \n")
            dic1 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic1['last_sign']
        if sign == last_sign and dic2 is None:  # Continue same trend
            dic1['ids'].append(idx)
            dic1['last_sign'] = sign
            dic1['cumulative'] *= (1 + log_ret)
            continue

        # 1st Reversal occuring
        if dic2 is None:  # Start dic2
            dic2 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic2['last_sign']
        if sign == last_sign and dic3 is None:  # Continue same trend
            dic2['ids'].append(idx)
            dic2['last_sign'] = sign
            dic2['cumulative'] *= (1 + log_ret)
            if len(dic2['ids']) == reverse_steps:
                assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                # print(f"dic1['cumulative'] = {dic1['cumulative']}, and dic1['ids'] = {dic1['ids']}")
                dic1 = dic2
                dic2 = None
                # print(f"dic1 after trend reversal persisted and dic1 = dic2 = \n{dic1}")
                # print(f"dic2 after being reset: {dic2}\n")
            continue

        # 2nd Reversal occuring
        if dic3 is None:  # Start dic3
            dic3 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic3['last_sign']
        if sign == last_sign: # Continue same trend, there is no dic4 to check if is None
            dic3['ids'].append(idx)
            dic3['last_sign'] = sign
            dic3['cumulative'] *= (1 + log_ret)
            dic_prod = dic2['cumulative'] * dic3['cumulative']
            # if (sign == 1 and dic1['cumulative'] * dic_prod > dic1['cumulative']) or (sign == -1 and dic1['cumulative'] * dic_prod < dic1['cumulative'])):
            if (sign == 1 and dic_prod > 1) or (sign == -1 and dic_prod < 1): # More beautiful
                # Merge dic1, dic2, and dic3
                dic1['ids'] += dic2['ids'] + dic3['ids']
                dic1['last_sign'] = dic3['last_sign']
                dic1['cumulative'] *= dic2['cumulative'] * dic3['cumulative']
                dic2, dic3 = None, None
                continue

            if len(dic3['ids']) == reverse_steps:      
                assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                assign_label(dic2, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                dic1 = dic3
                dic2, dic3 = None, None
                # print(f"dic2 after 2nd trend reversal didn't catch up fast enough, and now \ndic1 = dic3 = {dic1}")
                # print(f"dic3 and dic2 after being reset: {dic3}\n")
            continue
            
        # 3rd Reversal occuring
        assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
        # Reassign values
        dic1 = dic2
        dic2 = dic3
        dic3 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
        # print(f"There was a 3rd trend reversal, and now \ndic1 = dic2 = {dic1}, \ndic2 = dic3 = {dic2}")
        # print(f"dic3 after being reset: {dic3}\n")

    # Assign remaining labels
    if dic1:
        assign_label(dic1, lower_threshold, upper_threshold)
    if dic2:
        assign_label(dic2, lower_threshold, upper_threshold)
    if dic3:
        assign_label(dic3, lower_threshold, upper_threshold)
    # print("\n#-------------------- Returning 'trend' patterns ------------------------#")
    
    return df

def detect_trends_4(
    dataframe: pd.DataFrame, 
    column: str = 'close', 
    lower_threshold: float = 0.001, 
    upper_threshold: float = 0.02,
    reverse_steps: int = 7,
    trends_to_keep: set = {0, 1, 2, 3, 4}  # Default keeps all trends
) -> pd.DataFrame:
    """
    Detects trends based on log return data provided in a specified column and categorizes them into different strength levels.

    This function analyzes time-series data by evaluating cumulative trends in log return values provided in the input DataFrame. 
    It uses three dictionaries (`dic1`, `dic2`, `dic3`) to track different phases of trends, handles multi-step reversals, and 
    classifies trends dynamically based on cumulative product thresholds and specified thresholds for trend strengths.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing log return data.
        column (str): Column name containing log return values. Defaults to 'close'.
        lower_threshold (float): Threshold for categorizing moderate trends. Defaults to 0.001.
        upper_threshold (float): Threshold for categorizing strong trends. Defaults to 0.02.
        reverse_steps (int): Number of consecutive steps to confirm a trend reversal. Defaults to 7.
        trends_to_keep (set): A set of trend categories to retain; others will be set to 0 (No Trend). Defaults to keeping all trends {0, 1, 2, 3, 4}.

    Returns:
        pd.DataFrame: A DataFrame with an added column:
                    - 'trend': Categorized trend values based on the detected phases:
                        - 0: No trend
                        - 1: Moderate negative trend
                        - 2: Very strong negative trend
                        - 3: Moderate positive trend
                        - 4: Very strong positive trend
                      Any trends not included in `trends_to_keep` will be reset to 0.

    Function Details:
    1. **Input Assumption**:
    - The input DataFrame already contains log return data in the specified column (`column`).

    2. **Trend Tracking**:
    - Uses dictionaries to monitor trends:
        - `dic1`: Tracks the first phase of the trend.
        - `dic2`: Tracks the second phase if a reversal occurs.
        - `dic3`: Tracks the third phase if another reversal occurs.

    3. **Cumulative Product**:
    - Calculates the cumulative product of `(1 + log_return)` from the specified column to evaluate the strength of trends.

    4. **Reversal Handling**:
    - If a trend reversal persists beyond `reverse_steps`, labels are assigned based on the cumulative product tracked in `dic1`.
    - Subsequent reversals are merged or labeled independently if conditions are met.

    5. **Label Assignment**:
    - Labels are dynamically assigned based on cumulative product thresholds for positive and negative trends:
        - Positive trends are categorized as moderate (3) or strong (4).
        - Negative trends are categorized as moderate (1) or strong (2).

    6. **Trend Filtering**:
    - After detecting trends, only those specified in `trends_to_keep` remain unchanged.
    - Any trend category not included in `trends_to_keep` is reset to 0 (No Trend).

    7. **Edge Cases**:
    - Properly handles scenarios where data points are insufficient for trend analysis or when trend phases overlap, ensuring all data points are labeled.
    """
    # Copy to avoid modifying the original DataFrame
    df = dataframe.copy()
    df['trend'] = None  # Default value 
    
    # print("\n#-------------------- Working on 'trend' patterns -----------------------#")
    dic1, dic2, dic3 = None, None, None # Initialize trend tracking dictionaries
    # dic1 = None # {'ids': [], 'last_sign': None, 'cumulative': 1.0}
    
    def assign_label(dictio_, lower_threshold, upper_threshold):
        cumulative = dictio_['cumulative']
        # print(f"cumulative = {cumulative}")
        if cumulative > (1 + upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 4  # Very strong positive
        elif (1 + lower_threshold) < cumulative <= (1 + upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 3  # Moderate positive
        elif (1 - upper_threshold) < cumulative <= (1 - lower_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 1  # Moderate negative
        elif cumulative <= (1 - upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 2  # Very strong negative
        else:
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 0  # No trend
    
    #----------------------- For Loop -----------------------#
    for idx, log_ret in enumerate(df[column]):
        sign = 1 if log_ret > 0 else -1

        if dic1 is None:  # Initialize dic1
            # print(f"\nThis one time condition 'if loop' is running \n")
            dic1 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic1['last_sign']
        if sign == last_sign and dic2 is None:  # Continue same trend
            dic1['ids'].append(idx)
            dic1['last_sign'] = sign
            dic1['cumulative'] *= (1 + log_ret)
            continue

        # 1st Reversal occuring
        if dic2 is None:  # Start dic2
            dic2 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic2['last_sign']
        if sign == last_sign and dic3 is None:  # Continue same trend
            dic2['ids'].append(idx)
            dic2['last_sign'] = sign
            dic2['cumulative'] *= (1 + log_ret)
            if len(dic2['ids']) == reverse_steps:
                assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                # print(f"dic1['cumulative'] = {dic1['cumulative']}, and dic1['ids'] = {dic1['ids']}")
                dic1 = dic2
                dic2 = None
                # print(f"dic1 after trend reversal persisted and dic1 = dic2 = \n{dic1}")
                # print(f"dic2 after being reset: {dic2}\n")
            continue

        # 2nd Reversal occuring
        if dic3 is None:  # Start dic3
            dic3 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic3['last_sign']
        if sign == last_sign: # Continue same trend, there is no dic4 to check if is None
            dic3['ids'].append(idx)
            dic3['last_sign'] = sign
            dic3['cumulative'] *= (1 + log_ret)
            dic_prod = dic2['cumulative'] * dic3['cumulative']
            # if (sign == 1 and dic1['cumulative'] * dic_prod > dic1['cumulative']) or (sign == -1 and dic1['cumulative'] * dic_prod < dic1['cumulative'])):
            if (sign == 1 and dic_prod > 1) or (sign == -1 and dic_prod < 1): # More beautiful
                # Merge dic1, dic2, and dic3
                dic1['ids'] += dic2['ids'] + dic3['ids']
                dic1['last_sign'] = dic3['last_sign']
                dic1['cumulative'] *= dic2['cumulative'] * dic3['cumulative']
                dic2, dic3 = None, None
                continue

            if len(dic3['ids']) == reverse_steps:      
                assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                assign_label(dic2, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                dic1 = dic3
                dic2, dic3 = None, None
                # print(f"dic2 after 2nd trend reversal didn't catch up fast enough, and now \ndic1 = dic3 = {dic1}")
                # print(f"dic3 and dic2 after being reset: {dic3}\n")
            continue
            
        # 3rd Reversal occuring
        assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
        # Reassign values
        dic1 = dic2
        dic2 = dic3
        dic3 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
        # print(f"There was a 3rd trend reversal, and now \ndic1 = dic2 = {dic1}, \ndic2 = dic3 = {dic2}")
        # print(f"dic3 after being reset: {dic3}\n")

    # Assign remaining labels
    if dic1:
        assign_label(dic1, lower_threshold, upper_threshold)
    if dic2:
        assign_label(dic2, lower_threshold, upper_threshold)
    if dic3:
        assign_label(dic3, lower_threshold, upper_threshold)
    # print("\n#-------------------- Returning 'trend' patterns ------------------------#")
    
    # Apply filtering: Keep only selected trends, set others to 0
    df['trend'] = df['trend'].apply(lambda x: x if x in trends_to_keep else 0)

    return df

def split_X_y(sequences: list[pd.DataFrame], 
              target_column: str = 'trend',
              detect_trends_function: Callable[[pd.DataFrame, str, float, float, int, set], pd.DataFrame] = detect_trends_4, 
              column: str = 'close', 
              lower_threshold: float = 0.0009, 
              upper_threshold: float = 0.015,
              reverse_steps: int = 7,
              trends_to_keep: set = {0, 1, 2, 3, 4}) -> Tuple[np.ndarray, np.ndarray]:
    """
    Process sequences to generate features (X) and labels (y) while applying trend detection.

    Args:
    - sequences (list of pd.DataFrame): List of DataFrame sequences.
    - lower_threshold (float): Lower threshold for trend detection.
    - upper_threshold (float): Upper threshold for trend detection.
    - reverse_steps (int): Steps to reverse trends in the sequence.
    - target_column (str): Column name to use as the label (default: 'trend').
    - detect_trends_function (Callable): Function for detecting trends, defaults to `detect_trends_4`.
    - trends_to_keep (set): A set of trend categories to retain; others will be set to 0 (No Trend).

    Returns:
    - X (np.ndarray): Features array of shape (num_sequences, sequence_length, num_features).
    - y (np.ndarray): Labels array of shape (num_sequences,).
    """
    X = []
    y = []
    # count = 0
    for seq in sequences:
        # Apply trend detection on the sequence
        seq = detect_trends_function(seq, column=column, 
                                     lower_threshold=lower_threshold, 
                                     upper_threshold=upper_threshold, 
                                     reverse_steps=reverse_steps,
                                     trends_to_keep=trends_to_keep)
        # if count == 0:
        #     count = 1
        #     print(f"\nseq.head()")
        #     display(seq.head())
        #     print()
        # Extract features (X) and labels (y)
        X.append(seq.drop(columns=[target_column]).values)  # All but the target column
        y.append(seq[target_column].values)  # Target column as labels
        
    # return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)
    return np.array(X), np.array(y)

def relabel_split_data(
    data: pd.DataFrame, 
    detect_trends_function: Callable[[pd.DataFrame, str, float, float, int], pd.DataFrame], 
    column: str = 'close', 
    lower_threshold: float = 0.0009, 
    upper_threshold: float = 0.015,
    reverse_steps: int = 7
) -> pd.DataFrame:
    """
    Relabels the start and end of a DataFrame based on trend consistency.

    Args:
        data (pd.DataFrame): The DataFrame to relabel.
        detect_trends_function (Callable): A callable function to detect trends.
        column (str): Column name for trend detection. Defaults to 'close'.
        lower_threshold (float): Lower threshold for trend detection.
        upper_threshold (float): Upper threshold for trend detection.
        reverse_steps (int): Number of steps for reversal detection.

    Returns:
        pd.DataFrame: The relabeled DataFrame.
    """
    data = data.copy()

    # Get the start and end labels
    start_label = data['trend'].iloc[0]
    end_label = data['trend'].iloc[-1]

    # Identify start segment
    if start_label != 0:
        start_segment = []
        for idx, label in enumerate(data['trend']):
            if label == start_label:
                start_segment.append(idx)
            else:
                break
        start_segment = data.iloc[start_segment][[column]].copy()
        print(f"Start segment's label is not 0, it is {start_label} and data: ")
        display(data.loc[start_segment.index])
        # Apply trend detection on the start and end segments and Update the 'trend' column for the start and end segments
        if not start_segment.empty:
            start_segment_relabel = detect_trends_function(start_segment, column, lower_threshold, upper_threshold, reverse_steps)
            data.loc[start_segment.index, 'trend'] = start_segment_relabel['trend']
        print("Start segment after new label added: ")
        display(data.loc[start_segment.index])
    else:
        print(f"The start segment has a label of '{start_label}', so no operation needed. ")

    # Identify end segment
    if end_label != 0:
        end_segment = []
        for idx in range(len(data) - 1, -1, -1):
            if data['trend'].iloc[idx] == end_label:
                end_segment.append(idx)
            else:
                break
        end_segment = data.iloc[sorted(end_segment)][[column]].copy()
        print(f"End segment's label is not 0, it is {end_label} and data: ")
        display(data.loc[end_segment.index])
        # Apply trend detection on the start and end segments and Update the 'trend' column for the start and end segments
        if not end_segment.empty:
            end_segment_relabel = detect_trends_function(end_segment, column, lower_threshold, upper_threshold, reverse_steps)
            data.loc[end_segment.index, 'trend'] = end_segment_relabel['trend']
        print("End segment after new label added: ")
        display(data.loc[end_segment.index])
    else:
        print(f"The end segment has a label of '{end_label}', so no operation needed. ")

    return data

def process_and_return_splits(
    with_indicators_file_path: str,
    downsampled_data_minutes: int,
    exclude_columns: list[str],
    lower_threshold: float,
    upper_threshold: float,
    reverse_steps: int,
    sequence_length: int,
    sliding_interval: int,
    trends_to_keep: set = {0, 1, 2, 3, 4}  # Default keeps all trends
) -> tuple[
    list[list[float]],  # X_train: List of sequences, each containing a list of features
    list[list[int]],    # y_train: List of sequences, each containing a list of labels
    list[list[float]],  # X_val: List of sequences, each containing a list of features
    list[list[int]],    # y_val: List of sequences, each containing a list of labels
    list[list[float]],  # X_test: List of sequences, each containing a list of features
    list[list[int]]     # y_test: List of sequences, each containing a list of labels
]:
    """
    Processes time-series data from a CSV file and prepares it for machine learning.

    This function performs the following steps:
        1. Reads data from the specified CSV file and sorts it by date in descending order.
        2. Optionally downsamples the data to a lower frequency (e.g., 5-minute intervals).
        3. Applies Gaussian smoothing to reduce noise in the data.
        4. Calculates log returns for all numeric columns, excluding specified columns.
        5. Detects trends based on defined thresholds (`lower_threshold`, `upper_threshold`, and `reverse_steps`).
        6. Filters trends to keep only those specified in `trends_to_keep`, setting others to 0 (No Trend).
        7. Converts the processed data into sequences of a fixed length (`sequence_length`) with a sliding interval.
        8. Splits the sequences into training (80%), validation (10%), and test (10%) sets.
        9. Further splits the sequences into features (`X`) and labels (`y`) for supervised learning.

    Args:
        with_indicators_file_path (str): Path to the CSV file containing the time-series data.
        downsampled_data_minutes (int): Frequency for downsampling the data (e.g., 1 for no downsampling).
        exclude_columns (list[str]): List of column names to exclude from log return calculations.
        lower_threshold (float): Lower threshold for trend detection.
        upper_threshold (float): Upper threshold for trend detection.
        reverse_steps (int): Number of steps for reversing trends in trend detection.
        sequence_length (int): Length of sequences to create from the data.
        sliding_interval (int): Interval for sliding the window when creating sequences.
        trends_to_keep (set): A set of trend categories to retain; others will be set to 0 (No Trend). Defaults to keeping all trends {0, 1, 2, 3, 4}.

    Returns:
        tuple[list[list[float]], list[list[int]], list[list[float]], list[list[int]], list[list[float]], list[list[int]]]:
            A tuple containing:
            - X_train (list[list[float]]): Input sequences for training.
            - y_train (list[list[int]]): Target sequences for training.
            - X_val (list[list[float]]): Input sequences for validation.
            - y_val (list[list[int]]): Target sequences for validation.
            - X_test (list[list[float]]): Input sequences for testing.
            - y_test (list[list[int]]): Target sequences for testing.

    Example:
        X_train, y_train, X_val, y_val, X_test, y_test = process_and_return_splits(
            with_indicators_file_path="data.csv",
            downsampled_data_minutes=5,
            exclude_columns=["volume"],
            lower_threshold=-0.05,
            upper_threshold=0.05,
            reverse_steps=3,
            sequence_length=50,
            sliding_interval=5,
            trends_to_keep={1, 2, 3, 4}  # Only keep categorized trends, set others to 0
        )
    """

    data_retrieved = read_csv_file(with_indicators_file_path, preview_rows=0) # 190 days of data
    data_retrieved = data_retrieved.sort_index(ascending=False)

    #---------------------------------------------------------------------------------------
    if downsampled_data_minutes != 1:
        print("Downsampling the data! \n")
        data_retrieved = downsample_minute_data(data_retrieved, downsampled_data_minutes)
    #---------------------------------------------------------------------------------------

    # Get missing timestamps
    missing_timestamps = pd.date_range(
        start=data_retrieved.index.min(), # Returns smallest/earliest/oldest date
        end=data_retrieved.index.max(),
        freq='1min',  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
        tz=data_retrieved.index.tz,
    ).difference(data_retrieved.index)
    print(f"\ndata_retrieved - Missing timestamps time: \n{missing_timestamps}") 

    data_gaussian = gaussian_smoothing(data_retrieved, sigma=7)

    # Get missing timestamps
    missing_timestamps = pd.date_range(
        start=data_gaussian.index.min(), # Returns smallest/earliest/oldest date
        end=data_gaussian.index.max(),
        freq='1min',  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
        tz=data_gaussian.index.tz,
    ).difference(data_gaussian.index)
    print(f"\ndata_gaussian - Missing timestamps time: \n{missing_timestamps}\n")

    data_log_return = calculate_log_returns_all_columns(data_gaussian, exclude_columns=exclude_columns)

    # Get missing timestamps
    missing_timestamps = pd.date_range(
        start=data_log_return.index.min(), # Returns smallest/earliest/oldest date
        end=data_log_return.index.max(),
        freq='1min',  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
        tz=data_log_return.index.tz,
    ).difference(data_log_return.index)
    print(f"\ndata_log_return - Missing timestamps time: \n{missing_timestamps}\n") 

    # Check if there are missing timestamps
    if missing_timestamps.empty:
        print("No missing timestamps.")
    else:
        for timestamp in missing_timestamps:
            print(f"\nMissing timestamp: {timestamp}")
            
            # Create a placeholder for the missing timestamp
            if timestamp not in data_log_return.index:
                print('Missing')
            
            # Get data before and after the missing timestamp
            before_data = data_log_return[data_log_return.index < timestamp].tail(5)  # 5 data points before
            after_data = data_log_return[data_log_return.index > timestamp].head(5)  # 5 data points after
            
            # Display surrounding data
            if not before_data.empty:
                print("\nData before:")
                print(before_data)
            else:
                print("\nNo data available before the missing timestamp.")
            
            if not after_data.empty:
                print("\nData after:")
                print(after_data)
            else:
                print("\nNo data available after the missing timestamp.")

    sequences = created_sequences_2(data_log_return, sequence_length, sliding_interval)

    # Split sequences into training, validation, and test sets
    train_size = int(len(sequences) * 0.8)
    val_size = int(len(sequences) * 0.1)

    train_sequences = sequences[:train_size]
    val_sequences = sequences[train_size:train_size + val_size]
    test_sequences = sequences[train_size + val_size:]

    print(f"""
    Number of sequences:
        - sequences[0].shape: {sequences[0].shape}
        - Total sequences: {len(sequences)}
        - Train sequences: {len(train_sequences)}
        - Validation sequences: {len(val_sequences)}
        - Test sequences: {len(test_sequences)}
    """)

    # Process train, validation, and test sets
    X_train, y_train = split_X_y(train_sequences, 
                                target_column='trend',
                                detect_trends_function = detect_trends_4,
                                column= 'close',
                                lower_threshold=lower_threshold, 
                                upper_threshold=upper_threshold, 
                                reverse_steps=reverse_steps,
                                trends_to_keep=trends_to_keep)

    X_val, y_val = split_X_y(val_sequences, 
                            target_column='trend',
                            detect_trends_function = detect_trends_4,
                            column= 'close',
                            lower_threshold=lower_threshold, 
                            upper_threshold=upper_threshold, 
                            reverse_steps=reverse_steps,
                            trends_to_keep=trends_to_keep)

    X_test, y_test = split_X_y(test_sequences, 
                            target_column='trend',
                            detect_trends_function = detect_trends_4,
                            column= 'close',
                            lower_threshold=lower_threshold, 
                            upper_threshold=upper_threshold, 
                            reverse_steps=reverse_steps,
                            trends_to_keep=trends_to_keep)

    # Checking X arrays
    for idx, seq in enumerate(X_train):  # Loop through sequences
        for sub_idx, feature_set in enumerate(seq):  # Loop through data points
            for feature_idx, feature in enumerate(feature_set):  # Loop through features
                if not isinstance(feature, (float, np.float32)):  # Check each feature
                    print(f"Unexpected type in X_train at sequence {idx}, data point {sub_idx}, feature {feature_idx}: {type(feature)}")

    # Checking y arrays
    for idx, seq in enumerate(y_train):  # Loop through sequences
        for sub_idx, label in enumerate(seq):  # Loop through data points (labels)
            if not isinstance(label, (int, np.int64)):  # Check each label
                print(f"Unexpected type in y_train at sequence {idx}, data point {sub_idx}: {type(label)}")

    # Checking X arrays
    for idx, seq in enumerate(X_val):  # Loop through sequences
        for sub_idx, feature_set in enumerate(seq):  # Loop through data points
            for feature_idx, feature in enumerate(feature_set):  # Loop through features
                if not isinstance(feature, (float, np.float32)):  # Check each feature
                    print(f"Unexpected type in X_val at sequence {idx}, data point {sub_idx}, feature {feature_idx}: {type(feature)}")
    # Checking y arrays
    for idx, seq in enumerate(y_val):  # Loop through sequences
        for sub_idx, label in enumerate(seq):  # Loop through data points (labels)
            if not isinstance(label, (int, np.int64)):  # Check each label
                print(f"Unexpected type in y_val at sequence {idx}, data point {sub_idx}: {type(label)}")

    # Checking X arrays
    for idx, seq in enumerate(X_test):  # Loop through sequences
        for sub_idx, feature_set in enumerate(seq):  # Loop through data points
            for feature_idx, feature in enumerate(feature_set):  # Loop through features
                if not isinstance(feature, (float, np.float32)):  # Check each feature
                    print(f"Unexpected type in X_test at sequence {idx}, data point {sub_idx}, feature {feature_idx}: {type(feature)}")
    # Checking y arrays
    for idx, seq in enumerate(y_test):  # Loop through sequences
        for sub_idx, label in enumerate(seq):  # Loop through data points (labels)
            if not isinstance(label, (int, np.int64)):  # Check each label
                print(f"Unexpected type in y_test at sequence {idx}, data point {sub_idx}: {type(label)}")

    if isinstance(y_train, np.ndarray) and y_train.dtype == np.object_:
        # Convert to numeric if needed
        y_train = np.array(y_train, dtype=np.int64)

    if isinstance(y_val, np.ndarray) and y_val.dtype == np.object_:
        # Convert to numeric if needed
        y_val = np.array(y_val, dtype=np.int64)

    if isinstance(y_test, np.ndarray) and y_test.dtype == np.object_:
        # Convert to numeric if needed
        y_test = np.array(y_test, dtype=np.int64)

    close_col_index = data_log_return.columns.get_loc('close') # 'date' is set as index so doesnt count as a column
    Number_features = X_train.shape[-1]
    print(f"close_col_index = {close_col_index}")
    print(f"Number_features = {Number_features}")

    return X_train, y_train, X_val, y_val, X_test, y_test, Number_features


## All(Initial) parameters here

In [ ]:
ticker = 'BTC-USD'
downsampled_data_minutes = 1

lower_threshold = 0.0009 
upper_threshold = 0.015
reverse_steps = 13

exclude_columns= ['MACD', 'MACD_signal', 'ROC_10', 'OBV', 'AD_Line']
strongly_correlated = ['close', 'open', 'SMA_5', 'high', 'low', 'EMA_10', 'SMA_10'] # Strongly correlated (correlation > 0.6)
moderately_correlated = ['BB_middle', 'BB_lower', 'BB_upper', 'RSI_14'] # Moderately correlated (correlation between 0.3 and 0.6)
weakly_correlated = ['SMA_50', 'volume', 'BBW', 'ATR_14'] # Weakly correlated or negligible (correlation <~ 0.3)
exclude_columns += weakly_correlated + moderately_correlated

sequence_length = 1000
sliding_interval = 60

## **_Build the GRU Model_**

### GPU details

In [24]:
# If not in Jupyter Notebook
# import subprocess
# result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
# print(result.stdout)

!nvidia-smi

Mon Feb 17 13:39:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   47C    P8              5W /  103W |    2376MiB /   8188MiB |     22%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Torch and CUDA check

In [25]:
# Check if GPU is available
gpu_available = torch.cuda.is_available()

print("torch.cuda.is_available()", gpu_available)

# If GPU is available, print additional information
if gpu_available:
    print("\nGPU Device Name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
    print("Total CUDA Cores:", torch.cuda.get_device_properties(0).multi_processor_count * 128)  # NVIDIA GPUs often have 128 cores/SM
    print("Current GPU Device:", torch.cuda.current_device())
else:
    print("No GPU detected.")


torch.cuda.is_available() True

GPU Device Name: NVIDIA GeForce RTX 4060 Laptop GPU
Number of GPUs: 1
Total CUDA Cores: 3072
Current GPU Device: 0


### Model building

#### Bi-Directional GRU with Attention

## Train the Model

### list_period_files_full_path

In [43]:
pair = 'BTCUSD'
file_name = f'Polygon_{pair}_4Y_1min'  # File name for saving data
# BASE_FOLDER_PATH = f"{Working_directory}/Data/{file_name}"
BASE_FOLDER_PATH = f"Data/{file_name}"
# folder_path=f'{BASE_FOLDER_PATH}/polygon_io/12_USD_Crypto_Pairs/{file_name}'
folder_path=f'{BASE_FOLDER_PATH}'
if not os.path.isdir(folder_path):
    raise FileNotFoundError(f"Directory '{folder_path}' does not exist.")
file_path=f'{folder_path}/{file_name}.csv'
number_days = 190
with_indicators_file_path = os.path.normpath(f'{folder_path}/_{file_name}_{number_days}_days_with_indicators.csv')

# LABORATORY\_Global_Pytorch\Continual_Learning\Data\Polygon_BTCUSD_4Y_1min

list_period_files_full_path = [
    # Period 1
    with_indicators_file_path,
    # Period 2
    os.path.normpath(f"{folder_path}/Polygon_BTCUSD_4Y_1min_190_days__2020-11-11__2021-05-20__with_indicators.csv"),
    # Period 3
    os.path.normpath(f"{folder_path}/Polygon_BTCUSD_4Y_1min_190_days__2021-05-20__2021-11-26__with_indicators.csv"),
    # Period 4
    os.path.normpath(f"{folder_path}/Polygon_BTCUSD_4Y_1min_190_days__2021-11-26__2022-06-04__with_indicators.csv"),
    # Period 5
    os.path.normpath(f"{folder_path}/Polygon_BTCUSD_4Y_1min_190_days__2022-06-04__2022-12-11__with_indicators.csv")
]

with_indicators_file_path

'Data\\Polygon_BTCUSD_4Y_1min\\_Polygon_BTCUSD_4Y_1min_190_days_with_indicators.csv'

In [46]:
for file in os.listdir(folder_path):
    print(f"Found file: {file}")

# Polygon_BTCUSD_4Y_1min_190_days_with_indicators.csv

Found file: Polygon_BTCUSD_4Y_1min.csv
Found file: Polygon_BTCUSD_4Y_1min.png
Found file: Polygon_BTCUSD_4Y_1min_190_days__2020-11-11__2021-05-20.png
Found file: Polygon_BTCUSD_4Y_1min_190_days__2020-11-11__2021-05-20__with_indicators.csv
Found file: Polygon_BTCUSD_4Y_1min_190_days__2021-05-20__2021-11-26.png
Found file: Polygon_BTCUSD_4Y_1min_190_days__2021-05-20__2021-11-26__with_indicators.csv
Found file: Polygon_BTCUSD_4Y_1min_190_days__2021-11-26__2022-06-04.png
Found file: Polygon_BTCUSD_4Y_1min_190_days__2021-11-26__2022-06-04__with_indicators.csv
Found file: Polygon_BTCUSD_4Y_1min_190_days__2022-06-04__2022-12-11.png
Found file: Polygon_BTCUSD_4Y_1min_190_days__2022-06-04__2022-12-11__with_indicators.csv
Found file: Polygon_BTCUSD_4Y_1min_525_days.png
Found file: Polygon_BTCUSD_4Y_1min_525_days_with_indicators.csv
Found file: _Polygon_BTCUSD_4Y_1min_190_days.png
Found file: _Polygon_BTCUSD_4Y_1min_190_days_with_indicators.csv


In [47]:
X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
    with_indicators_file_path = list_period_files_full_path[0],
    downsampled_data_minutes = downsampled_data_minutes,
    exclude_columns = exclude_columns,
    lower_threshold = lower_threshold,
    upper_threshold = upper_threshold,
    reverse_steps = reverse_steps,
    sequence_length = sequence_length,
    sliding_interval = sliding_interval,
    trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
)

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

del X_train, y_train, X_val, y_val, X_test, y_test, Number_features
del unique_classes, num_classes



File path: Data\Polygon_BTCUSD_4Y_1min\_Polygon_BTCUSD_4Y_1min_190_days_with_indicators.csv

data_retrieved - Missing timestamps time: 
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='min')

data_gaussian - Missing timestamps time: 
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='min')

columns_to_transform = 
Index(['open', 'high', 'low', 'close', 'SMA_5', 'SMA_10', 'EMA_10'], dtype='object'), 
len(columns_to_transform) = 7

data_log_return - Missing timestamps time: 
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='min')

No missing timestamps.

    Number of sequences:
        - sequences[0].shape: (1000, 7)
        - Total sequences: 4543
        - Train sequences: 3634
        - Validation sequences: 454
        - Test sequences: 455
    
close_col_index = 3
Number_features = 7
unique_classes = [0 1 2 3 4]
num_classes = 5


### __*All periods data*__

In [ ]:
"""
- 'trend': Categorized trend values based on the detected phases:
    - 0: No trend
    - 1: Moderate negative trend
    - 2: Very strong negative trend
    - 3: Moderate positive trend
    - 4: Very strong positive trend
"""

# Initialize empty lists to combine elements
X_train_all = []
y_train_all = []
X_val_all = []
y_val_all = []
X_test_all = []
y_test_all = []
# for path_ in list_period_files_full_path:
#     with contextlib.redirect_stdout(open(os.devnull, 'w')):
#         X_train_, y_train_, X_val_, y_val_, X_test_, y_test_, Number_features = process_and_return_splits(
#             with_indicators_file_path = path_,
#             downsampled_data_minutes = downsampled_data_minutes,
#             exclude_columns = exclude_columns,
#             lower_threshold = lower_threshold,
#             upper_threshold = upper_threshold,
#             reverse_steps = reverse_steps,
#             sequence_length = sequence_length,
#             sliding_interval = sliding_interval,
#             trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
#         )
#     # Combine validation elements
#     X_train_all.extend(X_train_)
#     y_train_all.extend(y_train_)
#     X_val_all.extend(X_val_)
#     y_val_all.extend(y_val_)
#     X_test_all.extend(X_test_)
#     y_test_all.extend(y_test_)
#     # Delete unused variables to save memory
#     del X_train_, y_train_, X_val_, y_val_, X_test_, y_test_

# X_train_all = np.array(X_train_all)
# y_train_all = np.array(y_train_all)
# X_val_all = np.array(X_val_all)
# y_val_all = np.array(y_val_all)
# X_test_all = np.array(X_test_all)
# y_test_all = np.array(y_test_all)

with contextlib.redirect_stdout(open(os.devnull, 'w')):
    X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
        with_indicators_file_path = list_period_files_full_path[0],
        downsampled_data_minutes = downsampled_data_minutes,
        exclude_columns = exclude_columns,
        lower_threshold = lower_threshold,
        upper_threshold = upper_threshold,
        reverse_steps = reverse_steps,
        sequence_length = sequence_length,
        sliding_interval = sliding_interval,
        trends_to_keep = {0, 1}  # Default keeps all trends : {0, 1, 2, 3, 4}
        # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    )
    
print(f"\nNumber_features = {Number_features}")

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")



Number_features = 7
unique_classes = [0 1]
num_classes = 2
